In [1]:
import numpy as np
import math

In [2]:
def prepare(emails):
    spam=0;ham=0
    labels=[]
    emails_list=[]
    for email in emails :
        label = email.split(" ")[1]
        labels.append(label)
        email=email.split(" ")[2:]
        email = get_word_dic(email)
        #email=np.array(email)
        #email=email.reshape(len(email)/2,2)
        emails_list.append(email)
    return labels,emails_list

In [3]:
def get_word_dic(emails):
    word_dic={}
    e=np.array(emails)
    e=e.reshape(len(emails)/2,2)
    for i,x in e:
        try:
            word_dic[i]=(word_dic[i])+int(x)
        except:

            word_dic[i]=int(x)
    return word_dic


In [4]:
def Naive_Baise_train(labels_train,emails_train):
    len_train= len(labels_train)
    hamcount=0;spamcount=0;
    for i in labels_train:
        if i=="ham":
            hamcount+=1
    spamcount=len_train-hamcount
    pspam=spamcount/(float)(len_train)
    pham=1-pspam
    mixed_dir_ham={}
    mixed_dir_spam={}
    for key,email_dir in enumerate(emails_train):
        for word,freq in (email_dir.items()):
            if labels_train[key]=="ham":
                try:
                    mixed_dir_ham[word]=mixed_dir_ham[word]+freq
                except:
                    mixed_dir_ham[word]=int(freq)
            else :
                try:
                    mixed_dir_spam[word]=mixed_dir_spam[word]+freq
                except:
                    mixed_dir_spam[word]=int(freq)
    return mixed_dir_ham,mixed_dir_spam,pham,pspam,len_train

In [5]:
def top_five(dic):
    top=sorted(dic.iteritems(), key=lambda (v,k): (k,v),reverse=True)[:5]  
    return top

In [16]:
def p_of_word(word,dic,name="ham",smooth=True):
    total_word = sum(dic.values())
    #total_word = len(dic)
    k=1000
    try:
        p=(dic[word]+1)/(np.float64(total_word+k))
    except:
        p=1/(float(total_word+k))
    #print "P(",word,"|",name,") = " ,p
    return np.log2(p)
def get_all_pr(dic,name="ham"):
    total_word = sum(dic.values())
    t=0
    for word in dic:
        p=p_of_word(word,dic)
        t=t+p
        print "P(",word,"|",name,") = " ,(p)
    return t
def predict(email,spam,ham,prior_spam,prior_ham):
    pins=np.log2(prior_spam)
    pinh=np.log2(prior_ham)
    for word,count in email.items():
        pins=pins+(int(count)*NB_word(word,spam))
        pinh=pinh+(int(count)*NB_word(word,ham))
        
    if pins > pinh:
        return "spam"
    return "ham"
def NB_word(word,dic):
    return p_of_word(word,dic)

In [17]:
with open("data/train") as f:
    emails = f.readlines()
labels_train,emails_train=prepare(emails)

with open("data/test") as f:
    emails = f.readlines()
labels_test,emails_test=prepare(emails)

len_test= len(labels_test)
len_train= len(labels_train)

In [18]:
ham_dic,spam_dic,pham,pspam,_=Naive_Baise_train(labels_train,emails_train)

In [19]:
pham

0.42633333333333334

In [20]:
pspam

0.5736666666666667

In [21]:
error = 0
for key,email in enumerate(emails_test):
    y= predict(email,spam_dic,ham_dic,pspam,pham)
    if labels_test[key] != y :
        #print key,labels_test[key] ,y
        error +=1
print "Accuracy on Test data :",((len_test-error)*100)/float(len_test)

Accuracy on Test data : 90.2


In [22]:
error = 0
for key,email in enumerate(emails_train):
    y= predict(email,spam_dic,ham_dic,pspam,pham)
    if labels_train[key] != y :
        #print key,labels_test[key] ,y
        error +=1
print "Accuracy on Train data :",((len_train-error)*100)/float(len_train)

Accuracy on Train data : 90.8222222222


In [13]:
sum(spam_dic.values())+sum(ham_dic.values())

3233189

In [14]:
top_five(spam_dic)

[('enron', 63263),
 ('a', 39120),
 ('corp', 36005),
 ('the', 35487),
 ('to', 32608)]

In [15]:
top_five(ham_dic)

[('aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa',
  69975),
 ('enron', 67010),
 ('the', 52367),
 ('to', 39990),
 ('a', 28002)]